In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [9]:
class CNN(nn.Module):
    def __init__(self, input_size, out, dropout=0.5):
        super(CNN, self).__init__()
        self.fsize = int(input_size[1]*input_size[0]/2**3)*128
        self.input = input_size[0]*input_size[1]
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, padding=1) #40
        self.conv2 = nn.Conv1d(32, 32, kernel_size=3, padding=1) # 
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, padding=1) # 
        self.conv4 = nn.Conv1d(64, 64, kernel_size=3, padding=1) # 
        self.conv5 = nn.Conv1d(64, 128, kernel_size=3, padding=1) # 
        self.conv6 = nn.Conv1d(128, 128, kernel_size=3, padding=1) # 
        self.pool = nn.MaxPool1d(2)
        self.fc1 = nn.Linear(int(input_size[1]*input_size[0]/2**3)*128, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, out)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = x.view(-1, 1, self.input)
        x = F.relu(self.conv1(x)) # 40
        x = F.relu(self.conv2(x)) # 40
        x = self.pool(x) # 20
        x = F.relu(self.conv3(x)) # 20
        x = F.relu(self.conv4(x)) # 20
        x = self.pool(x) # 10
        x = F.relu(self.conv5(x)) # 10
        x = F.relu(self.conv6(x)) # 10
        x = self.pool(x) # 5*128
        x = x.view(-1, self.fsize) # 1*128*5
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x
# class CNN(nn.Module):
#     def __init__(self, input_size, out, dropout=0.5):
#         super(CNN, self).__init__()
#         self.fsize = int(input_size[1]*input_size[0]//2)*128
#         self.input = input_size[0]*input_size[1]
#         self.conv1 = nn.Conv1d(1, 32, kernel_size=3, padding=1) #40
#         self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1) # 
#         self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1) # 
#         self.pool = nn.MaxPool1d(2)
#         self.fc1 = nn.Linear(int(input_size[1]*input_size[0]//2)*128, 1024)
#         self.fc2 = nn.Linear(1024, 512)
#         self.fc3 = nn.Linear(512, out)
#         self.dropout = nn.Dropout(dropout)
        
#     def forward(self, x):
#         x = x.view(-1, 1, self.input)
#         x = F.relu(self.conv1(x)) # 40
#         x = F.relu(self.conv2(x)) # 40
#         x = F.relu(self.conv3(x)) # 20
#         x = self.pool(x) # 10
#         x = x.view(-1, self.fsize) # 1*128*5
#         x = F.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = F.relu(self.fc2(x))
#         x = self.dropout(x)
#         x = self.fc3(x)
#         return x

In [4]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len, output_dim, layers, drop_rate=0.5):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
        self.output_dim = output_dim
        self.layers = layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=layers,
                            dropout = drop_rate,
                            batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim, bias = True) 
        
    # 학습 초기화
    def reset_hidden_state(self): 
        self.hidden = (
                torch.zeros(self.layers, self.seq_len, self.hidden_dim),
                torch.zeros(self.layers, self.seq_len, self.hidden_dim))
    
    def forward(self, x, days):
        x, _status = self.lstm(x)
        x = self.fc(x[:, -1:])
        x = x.view(-1, days)
        return x

In [5]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_dim, dropout_prob):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, output_dim)
        
    def forward(self, x, days):
        out, _ = self.gru(x)
        out = self.fc(out[:, -days])
        out = out.view(-1, days)
        return out

In [7]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_dim, dropout_prob):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, output_dim)
        
    def forward(self, x, days):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -days])
        out = out.view(-1, days)
        return out